In [1]:
import numpy as np
import pandas as pd
from main import MLP_Classifier,Layer
from sklearn.datasets import make_classification
import optuna
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.model_selection import train_test_split

/opt/python/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(42)
# Generate  dataset
X, Y = make_classification(
    n_samples=1000,     
    n_features=4,       
    n_redundant=0,      
    n_clusters_per_class=1,
    flip_y=0.1,         # Add label noise
    class_sep=1.0,      # Class separation
    n_classes=2,      # nb classes
)


noise = np.random.normal(0, 0.5, X.shape)
X = X + noise
X=pd.DataFrame(X)
Y=pd.Series(Y)
if len(np.unique(Y))==2:
   Y=pd.DataFrame(Y)
else: 
   Y=pd.get_dummies(Y).astype(int)


determine objective function to optimise :  minimise cross entropy (in pdf maximise log-likelihood).

for example we optimise over batch size, learning rate and dropout (one of the most important parameters in NN).

we could do also on layers but computantionally expensive for large datasets.


In [3]:



def objective(trial):
    # Define hyperparameter search space
    batch_size = trial.suggest_int("batch_size", 500, 800)
    alpha = trial.suggest_float("alpha", 0.01, 0.1)
    dropout_rate = trial.suggest_float("dropout", 0.5, 0.9)

    model = MLP_Classifier(
        (
            (
                Layer(
                    nb_neurons=20,
                    activation_function="relu",
                    regul=("l2", 0.1),
                    initial="he",
                    batchnorm=True
                ),
                Layer(
                    nb_neurons=10,
                    activation_function="relu",
                    regul=("l2", 0.1),
                    initial="he",
                ),
                Layer(
                    nb_neurons=30,
                    activation_function="relu",
                    regul=("dropout", dropout_rate),
                    initial="he",
                ),
            )
        ),
        max_iter=2000,
        thr=1e-5,
        alpha=alpha,
        seed=123,
        batch_size=batch_size,
        verbose=False,
        optim="adam"
    )

    model.train(X, Y)

    score = model.loss(Y,model.y_hat)  # need to do on val set

    return score


storage = "sqlite:///optuna_mlpsoftmax.db"
study = optuna.create_study(
    direction="minimize", study_name="MLP", storage=storage, load_if_exists=True
)  # 'minimize' for loss functions
study.optimize(objective, n_trials=1)

print("Best Hyperparameters:", study.best_params)


[I 2026-01-02 16:31:40,679] Using an existing study with name 'MLP' instead of creating a new one.


Don't forget to normalise input data and think about Batch normalisations


[I 2026-01-02 16:31:41,905] Trial 2 finished with value: 0.39353641929822664 and parameters: {'batch_size': 687, 'alpha': 0.08443465052128196, 'dropout': 0.7595544399417837}. Best is trial 1 with value: 0.3868044002410619.


Best Hyperparameters: {'batch_size': 757, 'alpha': 0.011780812577113267, 'dropout': 0.8375822042749481}


In [4]:
best_results={"best value" : study.best_trial.values,"params": study.best_trial.params}
best_results


{'best value': [0.3868044002410619],
 'params': {'batch_size': 757,
  'alpha': 0.011780812577113267,
  'dropout': 0.8375822042749481}}

run model on optimised parameters

In [5]:


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

model = MLP_Classifier(
    (
        (
            Layer(
                nb_neurons=20,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                batchnorm=True
         
            ),
            Layer(
                nb_neurons=10,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                
            ),
            Layer(
                nb_neurons=30,
                activation_function="relu",
                regul=("dropout", best_results["params"]["dropout"]),
                initial="he",
                
            ),
        )
    ),
    max_iter=2000,
    thr=1e-5,
    alpha=best_results["params"]["alpha"],
    seed=123,
    batch_size=best_results["params"]["batch_size"],
    verbose=True,
    optim="adam",
    nb_epochs_early_stopping=50
)

fct=accuracy_score

model.train(X_train, y_train,X_test,y_test,fct)

print(f"final {fct.__name__}", accuracy_score(model.predict(X_test), y_test))


-------------------------------------------------------------------------
iteration 0 : TRAIN accuracy_score  : 0.7306666666666667, loss : 0.6721635938899835
iteration 0 : TEST accuracy_score  : 0.768, loss : 0.6726893745359764
-------------------------------------------------------------------------
iteration 50 : TRAIN accuracy_score  : 0.7906666666666666, loss : 0.49132845994433305
iteration 50 : TEST accuracy_score  : 0.828, loss : 0.4822462920750237
-------------------------------------------------------------------------
iteration 100 : TRAIN accuracy_score  : 0.8093333333333333, loss : 0.44333826703358514
iteration 100 : TEST accuracy_score  : 0.84, loss : 0.4162559547337287
-------------------------------------------------------------------------
iteration 150 : TRAIN accuracy_score  : 0.8093333333333333, loss : 0.4243432461689899
iteration 150 : TEST accuracy_score  : 0.848, loss : 0.3860931473604485
-------------------------------------------------------------------------
ite

In [10]:
model.network

{1: {'layer_type': layers.Layer,
  'nb_neurons': 20,
  'activ_fct': 'relu',
  'regul': 'l2',
  'regul_param': 0.1,
  'init': 'he',
  'law': 'normal',
  'batchnorm': True},
 2: {'layer_type': layers.Layer,
  'nb_neurons': 10,
  'activ_fct': 'relu',
  'regul': 'l2',
  'regul_param': 0.1,
  'init': 'he',
  'law': 'normal',
  'batchnorm': False},
 3: {'layer_type': layers.Layer,
  'nb_neurons': 30,
  'activ_fct': 'relu',
  'regul': 'dropout',
  'regul_param': 0.8375822042749481,
  'init': 'he',
  'law': 'normal',
  'batchnorm': False}}

In [1]:
from layers import ConvLayer,MaxPoolLayer,Layer,FlatLayer
from cnn import CNN


In [2]:

q=CNN(

    (
        ConvLayer(in_channels=3,output_channels=16,kernel_size=2,stride=1,padding=True,activation_function="relu",initial="lecun",law="normal"),
        MaxPoolLayer(kernel_size=3,stride=2,padding=True),
        ConvLayer(in_channels=16,output_channels=32,kernel_size=2,stride=1,padding=True,activation_function="relu",initial="lecun",law="normal"),
        MaxPoolLayer(kernel_size=3,stride=2,padding=True),
        ConvLayer(in_channels=32,output_channels=64,kernel_size=2,stride=1,padding=True,activation_function="relu",initial="lecun",law="normal"),
        MaxPoolLayer(kernel_size=3,stride=2,padding=True),
        FlatLayer(),
        Layer(
                nb_neurons=20,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                batchnorm=True
         
            ),
            Layer(
                nb_neurons=10,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                
            ),
            Layer(
                nb_neurons=30,
                activation_function="relu",
                regul=("dropout",0.5),
                initial="he",
                
            ),
        
    ),
    max_iter=2000,
    thr=1e-5,
    alpha=0.001,
    seed=123,
    batch_size=200,
    verbose=True,
    nb_epochs_early_stopping=20



)

Don't forget to normalise input data and think about Batch normalisations


In [4]:
import numpy as np 
X=np.random.randn(7, 7, 3,10)

q.train(X).shape

(7, 7, 3, 10)
(7, 7, 16, 10)
(4, 4, 16, 10)
(4, 4, 32, 10)
(2, 2, 32, 10)
(2, 2, 64, 10)
(1, 1, 64, 10)
(1, 1, 64, 10)


(10, 64)

In [7]:
q.ACV.keys()

dict_keys([1, 3, 5])

In [ ]:
q.nb_cnn_layers+q.nb_layers#total

from q.nb_cnn_layers+1 to total+1 do gradients mlp 


from 1 to q.nb_cnn_layers+1 do gradients cnn 

8

In [35]:

conv = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=2)
print(conv.weight.shape)
torch.Size([1, 1, 2, 2])

print(nn.init._calculate_fan_in_and_fan_out(conv.weight))


torch.Size([16, 3, 2, 2])
(12, 64)


In [19]:
fanin=3*3*3
FANOUT=3*3*16
fanin,FANOUT

(27, 144)

In [30]:
from torch import nn